In [1]:
import matplotlib.pyplot as plt
import math
import numpy
import pandas as pd
import pickle
import time


import src.tools as tools
import src.recencytools as recency
import src.postprocess as postprocess
import src.preprocess as preprocess
import src.tfidftools as tfidftools

%load_ext autoreload
%autoreload 2

## Load files

In [2]:
path_to_data = 'data/'

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)

training_info = pd.read_csv(
    path_to_data + 'training_info.csv', sep=',', parse_dates=True, header=0)

test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)

path_to_results = 'results/'

In [11]:
training_info = recency.add_time_rank_to_dataframe(training_info)
training_email_ids_per_sender = preprocess.get_email_ids_per_sender(training)

Beta is the coefficient that regulates time discrepancy.
The higher beta is, the more importance is given to older messages.

Each mail is weighted according to its rank (order of anciency).

In [13]:
beta = 2.1*math.pow(10,6)
recency_address_books = recency.get_recency_address_books(training_email_ids_per_sender,
                                                          training_info, beta, use_rank=False)
recency_predictions = recency.predictions_from_addressbook(training_email_ids_per_sender, recency_address_books)

Compute the recency address book.

In [17]:
# Write recency predictions to file
test_email_ids_per_sender = preprocess.get_email_ids_per_sender(test)
recency_predictions = recency.predictions_from_addressbook(test_email_ids_per_sender, recency_address_books)
recency_predictions = recency.recency_predictions_to_standard(recency_predictions)
postprocess.write_results_ranked(recency_predictions, path_to_results,
                                 'predictions_recency_time_{beta}.txt'.format(beta=beta))

In [18]:
# Model for fusion
test_email_ids_per_sender = preprocess.get_email_ids_per_sender(test)
recency_predictions = recency.predictions_from_addressbook(test_email_ids_per_sender, recency_address_books, keep_all=True)
recency_predictions = recency.recency_predictions_to_standard(recency_predictions)
path_to_ranks = 'ranks_test/recency_time_beta_{beta}_rec-all'.format(beta=beta)
with open(path_to_ranks, 'wb') as infile:
          pickle.dump(recency_predictions, infile)